# Workflow Test on Cosmos

### Testing the 'Sequence' Object in Python

In [2]:
#!/usr/bin/env python

import sys
sys.path.append('./modules') #to implement the modules in the 'modules' sub-directory
import Sequence
import tools
from Bio import SeqIO

sequences = []

for bioSeqRecord_obj in SeqIO.parse("data/test.fastq","fastq"):
    sequences.append(Sequence.Sequence(bioSeqRecord_obj))
print "BEFORE:"
print sequences[1].get_ID()
print sequences[1].get_SEQUENCE()
print sequences[1].get_LENGTH()


BEFORE:
SRR029695.24
AATCATCTTTCAAACCTATTGGTGCCTTTTCACCATATAATATATTGCCAACACACTAGCTAAAGCCATGAAA
73


In [3]:
print sequences[1].get_QUALITY()

[32, 26, 27, 25, 26, 27, 25, 32, 30, 12, 27, 33, 31, 17, 29, 22, 26, 24, 29, 22, 28, 21, 27, 27, 20, 10, 34, 32, 22, 10, 27, 27, 26, 18, 27, 27, 26, 27, 28, 21, 27, 27, 27, 27, 30, 23, 26, 24, 17, 29, 22, 27, 27, 26, 26, 27, 27, 27, 27, 27, 27, 33, 30, 12, 26, 27, 19, 27, 27, 27, 33, 30, 12]


**set_SEQUENCE()** function

In [4]:
sequences[1].set_SEQUENCE("AAAAAATTTTTAAAAGGGGCCCC")
print "AFTER:"
print sequences[1].get_SEQUENCE()
print sequences[1].get_LENGTH()

AFTER:
AAAAAATTTTTAAAAGGGGCCCC
23


**Quality treatment** test

In [5]:


tools.quality_treatment("data/test2.fastq","testing/data/quality_treatment_results.fastq",15)

Results of the Quality treatment tool in data/quality_treatment_results.fastq

**Length treatment** test

In [6]:

tools.length_treatment("data/quality_treatment_results.fastq","testing/data/length_treatment_results.fastq",60)

**Formating files** test

In [7]:
tools.format_files("data/quality_treatment_results.fastq","data/format_test.txt",'quality')
tools.format_files("data/quality_treatment_results.fastq","data/format_test2.txt",'length')


**Rscripts** test

In [8]:
tools.r_call("plot_quality_scores.R","data/format_test.txt","test.pdf")
tools.r_call("plot_sequence_length.R","data/format_test2.txt","test2.pdf")


### Implementation in Cosmos

In [9]:
#!/usr/bin/env python

import sys
sys.path.append('./modules') #to implement the modules in the 'modules' sub-directory
import Sequence
import tools
import os
import subprocess
from cosmos.api import Cosmos, draw_stage_graph, draw_task_graph, pygraphviz_available

def run_test(execution):
    
    QualTR = [execution.add_task(tools.quality_treatment,
                                 tags = dict(path2file=path2file, path2output="testing/data/quality_treatment_results.fastq", threshold=15)
                                 )
                for path2file in["testing/data/4.fastq"]]
    if pygraphviz_available:
        # These images can also be seen on the fly in the web-interface
        draw_stage_graph(execution.stage_graph(), '/tmp/test_task_graph.png', format='png')
        draw_task_graph(execution.task_graph(), '/tmp/test_stage_graph.png', format='png')
    else:
        print 'Pygraphviz is not available :('

    execution.run(max_attempts=1, max_cores=10)
   
cosmos = Cosmos('sqlite:///sqlite.db')
cosmos.initdb()

subprocess.check_call('mkdir -p testing testing/data', shell=True)
subprocess.check_call('cp extdata/4.fastq testing/data', shell=True)
execution = cosmos.start('Test1', 'testing',restart=True, skip_confirm=True)
run_test(execution)



Initializing sql database for Cosmos v1.7.3...
INFO: 2016-04-22 15:52:22: Deleting <Execution[1] Test1>, output_dir=/home/bulteaur/Documents/Modules/src/testing, delete_files=False
Deleting output_dir: /home/bulteaur/Documents/Modules/src/testing...
<Execution[1] Test1> Deleting from SQL...
<Execution[1] Test1> Deleted
INFO: 2016-04-22 15:52:23: Preparing to run <Execution[1] Test1> using DRM `local`, output_dir: `/home/bulteaur/Documents/Modules/src/testing`
INFO: 2016-04-22 15:52:23: <Stage[1] Quality_Treatment> Has not been attempted
INFO: 2016-04-22 15:52:23: Skipping 0 successful tasks...
INFO: 2016-04-22 15:52:23: Ensuring there are enough cores...
INFO: 2016-04-22 15:52:23: Committing to SQL db...
INFO: 2016-04-22 15:52:23: Executing TaskGraph
INFO: 2016-04-22 15:52:30: <Stage[1] Quality_Treatment> Running (1/1 tasks)
INFO: 2016-04-22 15:52:30: <Stage[1] Quality_Treatment> Finished successfully (1/1 successful tasks)
INFO: 2016-04-22 15:52:30: <Execution[1] Test1> Successfully F